### Importing Dataset and Dependencies

In [1]:
# Libraries Import
import pandas as pd
import ast
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Load your CSV
df = pd.read_csv("dataset.csv")
df.head()


c:\Users\Rajveer Mathur\Desktop\NER-BERT-finetuning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Template,Filled Template,Tokenised Filled Template,Tokens
0,"In our video conference, discuss the role of e...","In our video conference, discuss the role of e...","['in', 'our', 'video', 'conference', ',', 'dis...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,Could you draft a letter for [NAME_1] to send ...,"Could you draft a letter for Dietrich, Schulis...","['could', 'you', 'draft', 'a', 'letter', 'for'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-NAME', 'I-NA..."
2,Discuss the options for [FULLNAME_1] who wants...,Discuss the options for Jeffery Pfeffer who wa...,"['discuss', 'the', 'options', 'for', 'jeff', '...","['O', 'O', 'O', 'O', 'B-FULLNAME', 'I-FULLNAME..."
3,13. Write a press release announcing [FULLNAME...,13. Write a press release announcing Gayle Wat...,"['13', '.', 'write', 'a', 'press', 'release', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FULLNAM..."
4,9. Develop an inventory management plan for [F...,9. Develop an inventory management plan for Ev...,"['9', '.', 'develop', 'an', 'inventory', 'mana...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FU..."


### Preprocessing and Tokenization

In [2]:
# Function to convert string representation of lists into actual lists
def try_parse_list(cell):
    if isinstance(cell, str):
        return ast.literal_eval(cell)
    return cell

# This is necessary if the columns contain lists stored as strings
df["Tokenised Filled Template"] = df["Tokenised Filled Template"].apply(try_parse_list)
df["Tokens"] = df["Tokens"].apply(try_parse_list)
df.head()

,Template,Filled Template,Tokenised Filled Template,Tokens
0,"In our video conference, discuss the role of e...","In our video conference, discuss the role of e...","[in, our, video, conference, ,, discuss, the, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,Could you draft a letter for [NAME_1] to send ...,"Could you draft a letter for Dietrich, Schulis...","[could, you, draft, a, letter, for, dietrich, ...","[O, O, O, O, O, O, B-NAME, I-NAME, I-NAME, I-N..."
2,Discuss the options for [FULLNAME_1] who wants...,Discuss the options for Jeffery Pfeffer who wa...,"[discuss, the, options, for, jeff, ##ery, p, #...","[O, O, O, O, B-FULLNAME, I-FULLNAME, I-FULLNAM..."
3,13. Write a press release announcing [FULLNAME...,13. Write a press release announcing Gayle Wat...,"[13, ., write, a, press, release, announcing, ...","[O, O, O, O, O, O, O, B-FULLNAME, I-FULLNAME, ..."
4,9. Develop an inventory management plan for [F...,9. Develop an inventory management plan for Ev...,"[9, ., develop, an, inventory, management, pla...","[O, O, O, O, O, O, O, O, B-FULLNAME, I-FULLNAM..."


In [3]:
# Maximum number of tokens in any row
max_len = df["Tokenised Filled Template"].apply(len).max()
print(f"🔢 Maximum number of tokens in any row: {max_len}")

🔢 Maximum number of tokens in any row: 127


In [4]:
# Check for mismatches between tokens length and labels length
# This is to ensure that the tokenized filled template and tokens have the same length
counter = 0
for idx, row in df.iterrows():
    if len(row["Tokenised Filled Template"]) != len(row["Tokens"]):
        print(f"Mismatch at index {idx}")
        # print(f"Tokens ({len(row['Tokenised Filled Template'])}): {row['Tokenised Filled Template']}")
        # print(f"Labels ({len(row['Tokens'])}): {row['Tokens']}")
        counter += 1
print(f"Total mismatches found: {counter}")

Mismatch at index 701
Mismatch at index 1123
Mismatch at index 1906
Mismatch at index 2141
Mismatch at index 2192
Mismatch at index 2205
Mismatch at index 2375
Mismatch at index 2387
Mismatch at index 3218
Mismatch at index 3381
Mismatch at index 3654
Mismatch at index 3870
Mismatch at index 4265
Mismatch at index 4993
Mismatch at index 5071
Mismatch at index 5135
Mismatch at index 6312
Mismatch at index 6801
Mismatch at index 7190
Mismatch at index 7384
Mismatch at index 7779
Mismatch at index 7843
Mismatch at index 8652
Mismatch at index 8710
Mismatch at index 10344
Mismatch at index 10800
Mismatch at index 11358
Mismatch at index 12127
Mismatch at index 12358
Mismatch at index 13103
Mismatch at index 13467
Mismatch at index 13574
Mismatch at index 13642
Mismatch at index 13709
Mismatch at index 14074
Mismatch at index 14272
Mismatch at index 14604
Mismatch at index 15549
Mismatch at index 15775
Mismatch at index 15847
Mismatch at index 16239
Mismatch at index 16297
Mismatch at index

In [5]:
# Filter only rows where tokens and labels match in length
df = df[df["Tokenised Filled Template"].str.len() == df["Tokens"].str.len()]


# Check for mismatches between tokens and labels
counter = 0
for idx, row in df.iterrows():
    if len(row["Tokenised Filled Template"]) != len(row["Tokens"]):
        print(f"Mismatch at index {idx}")
        # print(f"Tokens ({len(row['Tokenised Filled Template'])}): {row['Tokenised Filled Template']}")
        # print(f"Labels ({len(row['Tokens'])}): {row['Tokens']}")
        counter += 1
print(f"Total mismatches found: {counter}")

Total mismatches found: 0


In [6]:
# Extract BIO tag scheme
all_tags = set(tag for tags in df["Tokens"] for tag in tags)
unique_tags = sorted(all_tags)

label2id = {tag: idx for idx, tag in enumerate(unique_tags)}
# print("Unique tags and their IDs:")
# for tag, idx in label2id.items():
#     print(f"{tag}: {idx}")

# Create a mapping from ID to label
id2label = {idx: tag for tag, idx in label2id.items()}
print("\nID to label mapping:")
for idx, tag in id2label.items():
    print(f"{idx}: {tag}")

# Add numeric label ids for each row
df["Label_ids"] = df["Tokens"].apply(lambda tags: [label2id[tag] for tag in tags])



ID to label mapping:
0: B-ACCOUNTNAME
1: B-ACCOUNTNUMBER
2: B-AMOUNT
3: B-BIC
4: B-BITCOINADDRESS
5: B-BUILDINGNUMBER
6: B-CITY
7: B-COUNTY
8: B-CREDITCARDCVV
9: B-CREDITCARDISSUER
10: B-CREDITCARDNUMBER
11: B-CURRENCY
12: B-CURRENCYCODE
13: B-CURRENCYNAME
14: B-CURRENCYSYMBOL
15: B-DISPLAYNAME
16: B-EMAIL
17: B-ETHEREUMADDRESS
18: B-FIRSTNAME
19: B-FULLNAME
20: B-GENDER
21: B-IBAN
22: B-IP
23: B-IPV4
24: B-IPV6
25: B-JOBAREA
26: B-JOBDESCRIPTOR
27: B-JOBTITLE
28: B-JOBTYPE
29: B-LASTNAME
30: B-LITECOINADDRESS
31: B-MAC
32: B-MASKEDNUMBER
33: B-NAME
34: B-NEARBYGPSCOORDINATE
35: B-NUMBER
36: B-ORDINALDIRECTION
37: B-PASSWORD
38: B-PIN
39: B-SECONDARYADDRESS
40: B-SEX
41: B-SEXTYPE
42: B-STATE
43: B-STREET
44: B-STREETADDRESS
45: B-URL
46: B-USERAGENT
47: B-USERNAME
48: B-ZIPCODE
49: I-ACCOUNTNAME
50: I-ACCOUNTNUMBER
51: I-AMOUNT
52: I-BIC
53: I-BITCOINADDRESS
54: I-BUILDINGNUMBER
55: I-CITY
56: I-CREDITCARDCVV
57: I-CREDITCARDISSUER
58: I-CREDITCARDNUMBER
59: I-CURRENCY
60: I-CURRENCY

In [7]:
# Train-validation split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
print(f"Training set size: {train_df.shape}")
print(f"Validation set size: {val_df.shape}")
# Convert to Hugging Face datasets
def to_hf_dataset(dataframe):
    return Dataset.from_dict({
        "tokens": dataframe["Tokenised Filled Template"].tolist(),
        "labels": dataframe["Label_ids"].tolist()
    })

train_dataset = to_hf_dataset(train_df)
val_dataset = to_hf_dataset(val_df)

Training set size: (20644, 5)
Validation set size: (2294, 5)


In [8]:
df.head()  # Display the first few rows of the DataFrame to verify changes

,Template,Filled Template,Tokenised Filled Template,Tokens,Label_ids
0,"In our video conference, discuss the role of e...","In our video conference, discuss the role of e...","[in, our, video, conference, ,, discuss, the, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 9..."
1,Could you draft a letter for [NAME_1] to send ...,"Could you draft a letter for Dietrich, Schulis...","[could, you, draft, a, letter, for, dietrich, ...","[O, O, O, O, O, O, B-NAME, I-NAME, I-NAME, I-N...","[92, 92, 92, 92, 92, 92, 33, 79, 79, 79, 79, 7..."
2,Discuss the options for [FULLNAME_1] who wants...,Discuss the options for Jeffery Pfeffer who wa...,"[discuss, the, options, for, jeff, ##ery, p, #...","[O, O, O, O, B-FULLNAME, I-FULLNAME, I-FULLNAM...","[92, 92, 92, 92, 19, 66, 66, 66, 66, 92, 92, 9..."
3,13. Write a press release announcing [FULLNAME...,13. Write a press release announcing Gayle Wat...,"[13, ., write, a, press, release, announcing, ...","[O, O, O, O, O, O, O, B-FULLNAME, I-FULLNAME, ...","[92, 92, 92, 92, 92, 92, 92, 19, 66, 66, 92, 9..."
4,9. Develop an inventory management plan for [F...,9. Develop an inventory management plan for Ev...,"[9, ., develop, an, inventory, management, pla...","[O, O, O, O, O, O, O, O, B-FULLNAME, I-FULLNAM...","[92, 92, 92, 92, 92, 92, 92, 92, 19, 66, 92, 9..."


In [9]:
# Target Label Encoding
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",   # ensure uniform tensor sizes
        max_length=128,
        return_attention_mask=True
    )

    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                # For subword tokens, assign -100 or the same label as the word
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

### Tokenizer Selection


In [10]:
from transformers import BertTokenizerFast, BertForTokenClassification

model_checkpoint = "prajjwal1/bert-tiny"
# model_checkpoint = "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

model = BertForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(unique_tags),
    id2label=id2label,
    label2id=label2id
)

train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
val_dataset = val_dataset.map(tokenize_and_align_labels, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

W0803 14:51:42.550000 18780 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
c:\Users\Rajveer Mathur\Desktop\NER-BERT-finetuning\.venv\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 2294/2294 [00:00<00:00, 5212.34 examples/s]


In [ ]:
# Trainer and Training Arguments
from transformers import TrainingArguments, Trainer
import numpy as np
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(predictions_output):
    preds, labels = predictions_output
    preds = np.argmax(preds, axis=2)

    true_preds = []
    true_labels = []

    for pred_seq, label_seq in zip(preds, labels):
        pred_tags = []
        label_tags = []
        for p, l in zip(pred_seq, label_seq):
            if l != -100:
                pred_tags.append(id2label[p])
                label_tags.append(id2label[l])
        true_preds.append(pred_tags)
        true_labels.append(label_tags)

    return {
        "accuracy": accuracy_score(true_labels, true_preds),
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
    }

# Training Argument
training_args = TrainingArguments(
    output_dir="./ner_output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
)


In [12]:
# Trainer start
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print("📈 Evaluation Metrics:")
for key, value in metrics.items():
    print(f"{key}: {value:.4f}")



  0%|          | 0/25810 [00:00<?, ?it/s]c:\Users\Rajveer Mathur\Desktop\NER-BERT-finetuning\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
  2%|▏         | 501/25810 [00:44<36:18, 11.62it/s]

{'loss': 2.3922, 'grad_norm': 3.694591760635376, 'learning_rate': 1.9612553273924838e-05, 'epoch': 0.19}


  4%|▍         | 1001/25810 [01:31<36:03, 11.47it/s]

{'loss': 1.116, 'grad_norm': 1.670870304107666, 'learning_rate': 1.922510654784967e-05, 'epoch': 0.39}


  6%|▌         | 1501/25810 [02:18<39:54, 10.15it/s]

...

{'loss': 0.1517, 'grad_norm': 0.9262222051620483, 'learning_rate': 2.402169701666021e-07, 'epoch': 9.88}


                                                     
100%|██████████| 25810/25810 [38:20<00:00, 11.22it/s]
c:\Users\Rajveer Mathur\Desktop\NER-BERT-finetuning\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.1283436119556427, 'eval_accuracy': 0.9682115467613958, 'eval_precision': 0.6833626982129374, 'eval_recall': 0.7880986937590712, 'eval_f1': 0.732003235373416, 'eval_runtime': 6.8195, 'eval_samples_per_second': 336.39, 'eval_steps_per_second': 42.085, 'epoch': 10.0}
{'train_runtime': 2300.1759, 'train_samples_per_second': 89.75, 'train_steps_per_second': 11.221, 'train_loss': 0.30106123488617237, 'epoch': 10.0}


100%|██████████| 287/287 [00:06<00:00, 42.83it/s]

📈 Evaluation Metrics:
eval_loss: 0.1283
eval_accuracy: 0.9682
eval_precision: 0.6834
eval_recall: 0.7881
eval_f1: 0.7320
eval_runtime: 6.7172
eval_samples_per_second: 341.5090
eval_steps_per_second: 42.7260
epoch: 10.0000


### Model Saving

In [13]:
# Model Saving
save_directory = f"./final_ner_model_{model_checkpoint.split('/')[-1]}"
print(f"Saving model to {save_directory}")

trainer.save_model(f"./final_ner_model_{model_checkpoint.split('/')[-1]}")
tokenizer.save_pretrained(f"./final_ner_model_{model_checkpoint.split('/')[-1]}")



Saving model to ./final_ner_model_bert-tiny


('./final_ner_model_bert-tiny\\tokenizer_config.json',
 './final_ner_model_bert-tiny\\special_tokens_map.json',
 './final_ner_model_bert-tiny\\vocab.txt',
 './final_ner_model_bert-tiny\\added_tokens.json',
 './final_ner_model_bert-tiny\\tokenizer.json')

### Model Inference

In [15]:
from transformers import pipeline

ner_pipeline = pipeline("ner", model=f"./final_ner_model_{model_checkpoint.split('/')[-1]}", tokenizer=f"./final_ner_model_{model_checkpoint.split('/')[-1]}", aggregation_strategy="simple")

text = "Dr. Marvin Rolfson and Julius Daugherty attended the arbitration. Their contact number is +91-8009992328"
print(ner_pipeline(text))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'FULLNAME', 'score': np.float32(0.94410837), 'word': 'dr. marvin rolfson', 'start': 0, 'end': 18}, {'entity_group': 'FULLNAME', 'score': np.float32(0.97039366), 'word': 'julius daugherty', 'start': 23, 'end': 39}, {'entity_group': 'NUMBER', 'score': np.float32(0.33254895), 'word': '91 - 80099', 'start': 91, 'end': 99}, {'entity_group': 'STREETADDRESS', 'score': np.float32(0.20343716), 'word': '##23', 'start': 100, 'end': 102}]
